# Find KEGG associations

This notebook will create a table that has the KEGG pathways that are associated with all genes, but we are particularly interested in those that are associated with the most and least stable genes.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import random
import pandas as pd
from scripts import paths, utils, modules, annotations

random.seed(1)

In [2]:
# Output files
pao1_out_filename = "pao1_core_similarity_associations.tsv"
pa14_out_filename = "pa14_core_similarity_associations.tsv"

In [3]:
# Load transcriptional similarity df
pao1_similarity_scores_filename = "pao1_core_similarity_expression_stats.tsv"
pa14_similarity_scores_filename = "pa14_core_similarity_expression_stats.tsv"

pao1_similarity_scores = pd.read_csv(
    pao1_similarity_scores_filename, sep="\t", header=0, index_col=0
)
pa14_similarity_scores = pd.read_csv(
    pa14_similarity_scores_filename, sep="\t", header=0, index_col=0
)

In [4]:
print(pao1_similarity_scores.shape)
pao1_similarity_scores.head()

(5349, 12)


,PA14 homolog id,Transcriptional similarity across strains,P-value,Name,label,comparison,median expression,mean expression,variance expression,max expression,min expression,range expression
PAO1 id,,,,,,,,,,,,
PA0984,PA14_59230,0.469691,9.403422e-292,NaN,NaN,NaN,62.749935,107.274133,16325.080802,16325.080802,0.057625,16325.023177
PA0680,PA14_55500,0.672592,0.000000e+00,hxcV,NaN,NaN,6.137738,12.204957,457.854387,467.814754,0.140207,467.674547
PA0847,PA14_53310,0.767508,0.000000e+00,NaN,NaN,NaN,187.968728,220.086486,16272.269787,16272.269787,36.746207,16235.523580
PA0164,PA14_02050,0.703763,0.000000e+00,NaN,NaN,NaN,107.041218,138.139287,14123.774503,14123.774503,10.463200,14113.311303
PA1160,PA14_49400,0.228437,2.819471e-64,NaN,NaN,NaN,137.427645,155.403740,10623.810909,10623.810909,1.905517,10621.905392


In [5]:
print(pa14_similarity_scores.shape)
pa14_similarity_scores.head()

(5348, 12)


,PAO1 homolog id,Transcriptional similarity across strains,P-value,Name,label,comparison,median expression,mean expression,variance expression,max expression,min expression,range expression
PA14 id,,,,,,,,,,,,
PA14_14200,PA3851,0.371554,1.018961e-174,NaN,NaN,NaN,108.816561,117.797345,3.195407e+03,3.195407e+03,10.148641,3.185258e+03
PA14_06250,PA0478,0.191827,1.697937e-45,NaN,least stable,least stable in both,49.936764,59.458793,2.266283e+03,2.266283e+03,4.448916,2.261834e+03
PA14_14160,PA3853,0.633848,0.000000e+00,NaN,NaN,NaN,123.889035,126.650667,2.384432e+03,2.384432e+03,19.859305,2.364573e+03
PA14_73300,PA5559,0.796787,0.000000e+00,atpE,NaN,most stable in SPELL,2567.030884,4982.538305,5.210635e+07,5.210635e+07,61.568086,5.210629e+07
PA14_62710,PA4740,0.849932,0.000000e+00,pnp,most stable,most stable in both,4189.606510,4940.105877,1.256453e+07,1.256453e+07,181.094326,1.256435e+07


In [6]:
# Load KEGG pathway data
pao1_pathway_filename = "https://raw.githubusercontent.com/greenelab/adage/7a4eda39d360b224268921dc1f2c14b32788ab16/Node_interpretation/pseudomonas_KEGG_terms.txt"

In [7]:
pao1_pathways = annotations.load_format_KEGG(pao1_pathway_filename)
pao1_pathways.head()

,1,2
0,,
KEGG-Pathway-pae00072: Synthesis and degradation of ketone bodies,10,"{PA3925, PA1736, PA2553, PA4785, PA2000, PA200..."
KEGG-Pathway-pae00071: Fatty acid degradation,32,"{PA4435, PA3629, PA2001, PA2815, PA0879, PA358..."
KEGG-Pathway-pae00903: Limonene and pinene degradation,9,"{PA3014, PA1821, PA4899, PA2475, PA1748, PA333..."
KEGG-Pathway-pae00380: Tryptophan metabolism,27,"{PA0202, PA4236, PA2001, PA2080, PA2579, PA358..."
KEGG-Pathway-pae00900: Terpenoid backbone biosynthesis,16,"{PA4044, PA3925, PA1736, PA2553, PA4557, PA478..."


## Pathway annotations to PA14

The annotations we have are only for PAO1 genes, so we will map PAO1 core genes to PA14 core genes to add annotations to PA14. This is possible since we are focused on only core genes, which have homologs between PAO1 and PA14

In [8]:
pao1_annotation_filename = paths.GENE_PAO1_ANNOT
gene_mapping_pao1 = utils.get_pao1_pa14_gene_map(pao1_annotation_filename, "pao1")

In [9]:
gene_mapping_pao1 = gene_mapping_pao1["PA14_ID"].to_frame()

## Get pathway associations for all genes

In [10]:
def get_associated_pathways(genes_):
    rows = []
    for gene_id in genes_:
        pathway_bool = [
            gene_id in pao1_pathways.loc[pathway, 2] for pathway in pao1_pathways.index
        ]
        found_pathways = list(pao1_pathways[pathway_bool].index)
        rows.append({"gene id": gene_id, "pathways present": found_pathways})
    return pd.DataFrame(rows).set_index("gene id")

In [11]:
# Get KEGG associations for all genes in PAO1
all_pao1_gene_ids = list(pao1_similarity_scores.index)
pao1_associations = get_associated_pathways(all_pao1_gene_ids)

In [12]:
print(pao1_associations.shape)
pao1_associations.head()

(5349, 1)


,pathways present
gene id,
PA0984,[]
PA0680,[KEGG-Pathway-pae03070: Bacterial secretion sy...
PA0847,[]
PA0164,[KEGG-Pathway-pae00480: Glutathione metabolism...
PA1160,[]


In [13]:
# Map PA14 gene ids
pa14_associations = pao1_associations.merge(
    gene_mapping_pao1, left_index=True, right_index=True
)
pa14_associations.set_index("PA14_ID", inplace=True)
print(pa14_associations.shape)
pa14_associations.head()

(5349, 1)


,pathways present
PA14_ID,
PA14_59230,[]
PA14_55500,[KEGG-Pathway-pae03070: Bacterial secretion sy...
PA14_53310,[]
PA14_02050,[KEGG-Pathway-pae00480: Glutathione metabolism...
PA14_49400,[]


In [14]:
# Merge KEGG associations with transcriptional similarity information
pao1_associations = pao1_similarity_scores.merge(
    pao1_associations, left_index=True, right_index=True, how="left"
)
pa14_associations = pa14_similarity_scores.merge(
    pa14_associations, left_index=True, right_index=True, how="left"
)

In [15]:
print(pao1_associations.shape)
pao1_associations.head()

(5349, 13)


,PA14 homolog id,Transcriptional similarity across strains,P-value,Name,label,comparison,median expression,mean expression,variance expression,max expression,min expression,range expression,pathways present
PAO1 id,,,,,,,,,,,,,
PA0984,PA14_59230,0.469691,9.403422e-292,NaN,NaN,NaN,62.749935,107.274133,16325.080802,16325.080802,0.057625,16325.023177,[]
PA0680,PA14_55500,0.672592,0.000000e+00,hxcV,NaN,NaN,6.137738,12.204957,457.854387,467.814754,0.140207,467.674547,[KEGG-Pathway-pae03070: Bacterial secretion sy...
PA0847,PA14_53310,0.767508,0.000000e+00,NaN,NaN,NaN,187.968728,220.086486,16272.269787,16272.269787,36.746207,16235.523580,[]
PA0164,PA14_02050,0.703763,0.000000e+00,NaN,NaN,NaN,107.041218,138.139287,14123.774503,14123.774503,10.463200,14113.311303,[KEGG-Pathway-pae00480: Glutathione metabolism...
PA1160,PA14_49400,0.228437,2.819471e-64,NaN,NaN,NaN,137.427645,155.403740,10623.810909,10623.810909,1.905517,10621.905392,[]


In [16]:
print(pa14_associations.shape)
pa14_associations.head()

(5348, 13)


,PAO1 homolog id,Transcriptional similarity across strains,P-value,Name,label,comparison,median expression,mean expression,variance expression,max expression,min expression,range expression,pathways present
PA14 id,,,,,,,,,,,,,
PA14_14200,PA3851,0.371554,1.018961e-174,NaN,NaN,NaN,108.816561,117.797345,3.195407e+03,3.195407e+03,10.148641,3.185258e+03,[]
PA14_06250,PA0478,0.191827,1.697937e-45,NaN,least stable,least stable in both,49.936764,59.458793,2.266283e+03,2.266283e+03,4.448916,2.261834e+03,[]
PA14_14160,PA3853,0.633848,0.000000e+00,NaN,NaN,NaN,123.889035,126.650667,2.384432e+03,2.384432e+03,19.859305,2.364573e+03,[]
PA14_73300,PA5559,0.796787,0.000000e+00,atpE,NaN,most stable in SPELL,2567.030884,4982.538305,5.210635e+07,5.210635e+07,61.568086,5.210629e+07,[KEGG-Pathway-pae00190: Oxidative phosphorylat...
PA14_62710,PA4740,0.849932,0.000000e+00,pnp,most stable,most stable in both,4189.606510,4940.105877,1.256453e+07,1.256453e+07,181.094326,1.256435e+07,"[KEGG-Pathway-pae00230: Purine metabolism, KEG..."


In [17]:
# Save
pao1_associations.to_csv(pao1_out_filename, sep="\t")
pa14_associations.to_csv(pa14_out_filename, sep="\t")

**Takeaway:**

Based on the pathways associated with the most and least stable core genes, we find that the most stable core genes tend to be associated with pathways related to cellular maintenance including: protein transport systems, ribosomes, metabolism, type III, IV secretion system which mediates virulence.

There are far fewer KEGG pathways that least stable core genes are found to be associated with. The least stable core genes are mostly associated with different types of metabolism.

A google doc containing the most and least stable core genes and some information about them is [here](https://docs.google.com/spreadsheets/d/1SqEyBvutfbsOTo4afg9GiEzP32ZKplkN1a6MpAQBvZI/edit?usp=sharing).